In [23]:
import re
import pdfplumber
import pandas as pd
from collections import namedtuple
#https://www.youtube.com/watch?v=syEfR1QIGcY

In [24]:
Line = namedtuple("Line","CID10 DESC JANEIRO FEVEREIRO MARÇO ABRIL MAIO JUNHO JULHO AGOSTO SETEMBRO OUTUBRO NOVEMBRO DEZEMBRO")
#Line = namedtuple("Line","JANEIRO FEVEREIRO MARÇO ABRIL MAIO JUNHO JULHO AGOSTO SETEMBRO OUTUBRO NOVEMBRO DEZEMBRO")
#Line = namedtuple("Line","CID10 DESC")


In [25]:
#line_sub = re.sub(r'[]','')
line_re =re.compile(r'(^\w{1}\d{2}[*†]*|[^\-\w]) ([\s\S]+(?<=[A-Za-z(),[\]\"áàâãéèêíïóôõöúçñÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ])) ([\s\S]+(?![0-9 -]))', flags=2)

#line_re.search('A15 Tuberculose respiratória, com confirmação bacteriológica e his 47 49 42 29 36 36 41 26 38 35 32 37').group(3)
#texto ="A15 Tuberculose respiratória, com confirmação bacteriológica e his 47 49 42 29 36 36 41 26 38 35 32 37"
#texto ="A17† Tuberculose do sistema nervoso 1 – – 1 –––3 – – – –"
#texto="M54 Dorsalgia 3.498 3.353 3.916 4.418 3.951 3.829 3.896 3.795 4.595 4.613 3.940 3.439"
texto="– Ignorado 27 24 36 35 27 45 34 59 42 37 44 44"
texto="Capítulo I: Algumas doenças infecciosas e parasitárias (A00-B99) 176 183 200 209 174 171 165 204 241 221 183 159"
#texto="I00-I02 Febre reumática aguda – ––––––– –– – –"
print((line_re.search(texto).group(1)))
print((line_re.search(texto).group(2)))
print((line_re.search(texto).group(3)))
comp =line_re.search(texto)
print(Line)
CID, CID_DES, DADOS = comp.group(1), comp.group(2) , comp.group(3)
DADOS = DADOS.replace(".","")
items = DADOS.split()
print((items))

#line_re.search('G35-G37 Doenças desmielinizantes do sistema nervoso central').group(1)

:
Algumas doenças infecciosas e parasitárias (A00-B99)
176 183 200 209 174 171 165 204 241 221 183 159
<class '__main__.Line'>
['176', '183', '200', '209', '174', '171', '165', '204', '241', '221', '183', '159']


In [26]:
cNome='2006_Previdenciarios'
file='data//'+cNome+'.pdf'
print(file)

data//2006_Previdenciarios.pdf


In [16]:
lines =[]
total_check =0 
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text(x_tolerance=5, y_tolerance=2)
        for line in text.split('\n'):
            comp =line_re.search(line)
            if comp:
                CID, CID_DES, DADOS = comp.group(1), comp.group(2) , comp.group(3)
                #CID_DES = '"'+CID_DES+'"'
                #linha = re.sub('(?<=.)(?!$)',' ',DADOS)
                DADOS = DADOS.replace(".","")
                DADOS = re.sub(r'(\d)\s(\d)', r'\1\2', DADOS)
                items = DADOS.split()
                if(CID=='–'):
                    print(line)
                    print(DADOS)
                    print(items)
                if(len(items)==12):
                    #print(Line(CID,CID_DES,*items))
                    lines.append(Line(CID,CID_DES,*items))

Acompanhamento Mensal dos Benefícios Auxílios-Doença Previdenciários Concedidos segundo os Códigos da CID-10 – Ano 2006
2006
['2006']
– Ignorado      6.138      6.145      7.597     6 .737      8.493      7.871      7.928   1 0.707      9 .433   1 0.768    1 0.591       9 .928
     6138      6145      7597     6737      8493      7871      7928   10707      9433   10768    10591       9928
['6138', '6145', '7597', '6737', '8493', '7871', '7928', '10707', '9433', '10768', '10591', '9928']
Acompanhamento Mensal dos Benefícios Auxílios-Doença Acidentários Concedidos segundo os Códigos da CID-10 – Ano 2006
2006
['2006']
– Ignorado    1.288     1 .119    1.303   1 .165   1 .428   1 .409   1 .299   1 .654      1 .351    1.790      1 .491       1 .320
   1288     1119    1303   1165   1428   1409   1299   1654      1351    1790      1491       1320
['1288', '1119', '1303', '1165', '1428', '1409', '1299', '1654', '1351', '1790', '1491', '1320']
– Ano 2 Setembr 10.58                            

In [17]:
dfCat =  pd.read_csv("data//CID-10-CAPITULOS.csv",delimiter=";",encoding='cp1252')
dfCat.drop(columns='Unnamed: 5',axis=1,inplace=True)
df = pd.DataFrame(lines)
#df.set_index('CID10',inplace=True)
cols = df.iloc[:,2:len(df)].columns
df.iloc[:,2:len(df)].replace("–",0,inplace=True)
df[cols] = df[cols].apply(pd.to_numeric,errors='coerce')
df.dtypes

CID10        object
DESC         object
JANEIRO       int64
FEVEREIRO     int64
MARÇO         int64
ABRIL         int64
MAIO          int64
JUNHO         int64
JULHO         int64
AGOSTO        int64
SETEMBRO      int64
OUTUBRO       int64
NOVEMBRO      int64
DEZEMBRO      int64
dtype: object

In [18]:
df = df.assign(key=1).merge(dfCat.assign(key=1), on='key')\
                 .drop(columns='key')\
                 .query("CID10.between(CATINIC,CATFIM)|CID10.str.contains('–')")\
                 .drop(columns=['CATINIC', 'CATFIM','DESCRICAO','DESCRABREV'])\
                 .reset_index(drop=True)
df.drop_duplicates(subset=['CID10','JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO'],keep='first',inplace=True)

In [19]:
df[df['NUMCAP'] ==2]['JANEIRO'].sum()


5762

In [20]:
df['SETEMBRO'].sum()

191250

In [21]:
df = df.append(df.sum(numeric_only=True), ignore_index=True)
df.to_excel(cNome+".xlsx",sheet_name='Acidentarios')
df.to_csv(cNome+".csv")

In [22]:
df

,CID10,DESC,JANEIRO,FEVEREIRO,MARÇO,ABRIL,MAIO,JUNHO,JULHO,AGOSTO,SETEMBRO,OUTUBRO,NOVEMBRO,DEZEMBRO,NUMCAP
0,A00,Cólera,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,A01,Febres tifóide e paratifóide,2.0,1.0,2.0,0.0,1.0,1.0,2.0,0.0,3.0,3.0,1.0,2.0,1.0
2,A02,Outras infecções por Salmonella,1.0,3.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,A03,Shiguelose,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,A04,Outras infecções intestinais bacterianas,4.0,4.0,0.0,4.0,2.0,4.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3146,Z54,Convalescença,16.0,10.0,4.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,21.0
3147,Z97,Presença de outros dispositivos protéticos,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
3148,Z99,Dependência de máquinas e dispositivos capacit...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
3149,–,Ignorado,1288.0,1119.0,1303.0,1165.0,1428.0,1409.0,1299.0,1654.0,1351.0,1790.0,1491.0,1320.0,1.0
